In [ ]:
# Display

import cv2
import matplotlib.pyplot as plt
from pathlib import Path

def plot_image_with_bboxes(image_path, label_path):
    """
    Display an image with bounding boxes drawn based on YOLO labels.

    Args:
    - image_path (str): Path to the image file.
    - label_path (str): Path to the YOLO label file corresponding to the image.
    """
    # Load the image
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for matplotlib

    # Get image dimensions
    height, width, _ = image.shape

    # Read YOLO label file
    with open(label_path, 'r') as f:
        labels = f.readlines()

    # Draw bounding boxes
    for label in labels:
        parts = label.strip().split()
        class_id, x_center, y_center, box_width, box_height = map(float, parts)

        # Denormalize the coordinates
        x_center *= width
        y_center *= height
        box_width *= width
        box_height *= height

        # Calculate top-left and bottom-right corners
        x_min = int(x_center - box_width / 2)
        y_min = int(y_center - box_height / 2)
        x_max = int(x_center + box_width / 2)
        y_max = int(y_center + box_height / 2)

        # Draw the bounding box
        color = (255, 0, 0)  # Red color for bounding boxes
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)
        cv2.putText(image, f'Class {int(class_id)}', (x_min, y_min - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the image
    plt.figure(figsize=(10, 6), dpi=600)
    plt.imshow(image)
    plt.axis('off')
    plt.show()

# Example usage
image_folder = Path("/content/drive/MyDrive/Colab Notebooks/project/data/train_images")
label_folder = Path("/content/drive/MyDrive/Colab Notebooks/project/data/labels/train")

# Get all image and label files
images = sorted(image_folder.glob("*.tif"))
labels = sorted(label_folder.glob("*.txt"))

# Display the first 5 images with bounding boxes
for img_path, lbl_path in zip(images[:5], labels[:5]):
    print(f"Displaying: {img_path.name}")
    plot_image_with_bboxes(img_path, lbl_path)

# saved on Github without output (too big pictures)

In [ ]:
import json
import pandas as pd
import tqdm

# Load GeoJSON file
geojson_file_path = '/content/drive/MyDrive/Colab Notebooks/project/data/xView_train.geojson'
with open(geojson_file_path, 'r') as f:
    geojson_data = json.load(f)

# Extract features
features = geojson_data['features']

# Convert features to a DataFrame
data = []
for feature in features:
    properties = feature['properties']
    data.append({
        'image_id': properties['image_id'],
        'class_id': properties['type_id'],
        'bounds_imcoords': properties['bounds_imcoords'],
        'additional_metadata': properties.get('metadata_field', None)  # Replace with actual metadata fields
    })

df = pd.DataFrame(data)

# Show the first few rows
print(df.head())